<a href="https://colab.research.google.com/github/limseo12/deep-learning-from-scratch_Study/blob/main/%EB%B0%91%EB%B0%94%EB%8B%A5%EB%B6%80%ED%84%B0%EC%8B%9C%EC%9E%91%ED%95%98%EB%8A%94%EB%94%A5%EB%9F%AC%EB%8B%9D3_%EC%A0%9C2%EA%B3%A0%EC%A7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step11 가변 길이 인수(순전파 편)

In [ ]:
#11.1 Function 클래스 수정
class Function:
  def __call__(self, input):
    x = input.data    #1 Variable 이라는 '상자'에서 실제 데이터를 꺼낸다
    y = self.forward(x)   #2 forward 메서드에서 구체적인 계산을 한다
    output = Variable(as_array(y))    #3 계산 결과를 Variable에 넣고
    output.set_creator(self)    #4 자신이 '창조자'라고 원산지 표시?를 한다.
    self.input = input
    self.output = output
    return output

  def forward(self, x):
    raise NotImplementedError()

  def backward(self, gy):
    raise NotImplementedError()

In [ ]:
#call 메서드의 인수와 반환값을 리스트로 바꿔보자.
class Function:
  def __call__(self, inputs):
    xs = [x.data for x in inputs]
    ys = self.forward(xs)
    output = [Variable(as_array(y)) for y in ys]

    for output in outputs:
      output.set_creator(self)
    self.inputs = inputs
    self.outputs = outputs
    return outputs

  def forward(self, xs):
    raise NotImplementedError()
  
  def backward(self, gys):
    raise NotImplementedError()

In [ ]:
#11.2 Add 클래스 구현
class Add(Function):
  def forward(self, xs):
    x0, x1 = xs
    y = x0 + x1
    return (y,)

In [ ]:
xs = [Variable(np.array(2)), Variable(np.array(3))] #리스트로 준비
f = Add()
ys = f(xs)
y = ys[0]
print(y.data)

# Step 12 가변 길이 인수(개선 편)

In [ ]:
#12.1 첫 번째 개선 : 함수를 사용하기 쉽게
class Function:
  def __call__(self, *inputs):  # 별표를 붙인다.
    xs = [x.data for x in inputs]
    ys = self.forward(xs)
    outputs = [Variable(as_array(y)) for y in ys]

    for output in outputs:
      output.set_creator(self)
    self.inputs = inputs
    self.outputs = outputs

    #리스트의 원소가 하나라면 첫 번째 원소를 반환한다.
    return outputs if len(outputs) > 1 else outputs[0]

In [ ]:
>>> def f(*x):
print(x)

>>> f(1, 2, 3)
(1, 2, 3)

>>> f(1, 2, 3, 4, 5, 6)
(1, 2, 3, 4, 5, 6)

In [ ]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
f = Add()
y = f(x0, x1)
print(y.data)

In [ ]:
# 12.2 두 번째 개선: 함수를 구현하기 쉽도록
class Function:
  def __call__(self, *inpuit):
    xs = [x.data for x in inputs]
    ys = self.forward(*xs)    # 1 별표를 붙여 언팩
    if not isinstance(ys, tuple):   # 2 튜플이 아닌 경우 추가 지원
      ys = (ys, )
    output = [Variable(as_array(y)) for y in ys]

    for output in outputs:
      output.set_creator(self)
    self.inputs = inputs
    self.inputs = outputs

    return outputs if len(outputs) > 1 else outputs[0]

In [ ]:
class Add(Function):
  def forward(self, x0, x1):
    y = x0 + x1
    return y

In [ ]:
#12.3 add 함수 구현
def add(x0, x1):
  return Add()(x0, x1)

In [ ]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)
print(y.data)

# Step13 가변 길이 인수(역전파 편)

In [ ]:
class Add(Function):
  def forward(self, x0, x1):
    y = x0 + x1
    return y
  
  def backward(self, gy):
    return gy, gy

In [ ]:
#13.2 Variable 클래스 수정
#Variable 클래스의 backward 메서드를 살펴보자
class Variable
##생략
def backward(self):
  if self.grad is None:
    self.grad = np.ones_like(delf.data)

  funcs = [self.creator]
  while funcs:
    f = funcs.pop()
    x, y = f.input, f.output # 1. 함수의 입출력을 얻는다.
    x.grad = f.backward(y.grad) # 1. backward 메서드를 호출한다.

    if x.creator is not None:
      funcs.append(x.creator)

In [ ]:
##
class Variable:
  #생략
  def backward(self):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = [self.creator]
    while funcs:
      f = funcs.pop()
      gys = [output.grad for output in f.outputs] # 1
      gxs = f.backward(*gys)  #2
      if not isinstance(gxs, tuple):  # 3
        gxs = (gxs, )
      
      for x, gx in zip(f.inputs, gxs):  # 4
        x.grad = gx

        if x.creator is not None:
          funcs.append(x.creator)

In [ ]:
#13.3 Square 클래스 구현
class Square(Function):
  def forward(self, x):
    y = x **2
    return y
  
  def backward(self, gy):
    x = self.inputs[0].data # 수정 전: x = self.input.data
    gx = 2 * x * gy
    return gx

In [ ]:
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = add(square(x), square(y))
z.backward()
print(z.data)
print(x.grad)
print(y.grad)

# Step 14 같은 변수 반복 사용

In [ ]:
#동일한 변수를 사용하여 덧셈을 하면 제대로 미분하지 못한다.
x = Variable(np.array(3.0))
y = add(x, x)
print('y', y.data)

y.backward()
print('x.grad', x.grad)

#결과
#y 6.0
#x.grad 1.0

#x = 3.0으로 설정한 후 계산하였을 때 y의 값은 6.0이라 제대로 계산하였지만
#미분값은 2 가된다. 원인은 Variable 클래스의 다음 위치에 있다.

In [ ]:
#14.1 문제의원인
class Variable
##생략
  def backward(self):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = [self.creator]
    while funcs:
      f = funcs.pop()
      gys = [output.grad for output in f.outputs]
      gxs = f.backward(*gys)
      if not isinstance(gxs, tuple):
        gxs = (gxs, )

      for x, gx in zip(f.inputs, gxs):
        x.grad = gx # 여기가 실수!

        if x.creator is not None:
          funcs.append(x.creator)

#미분값이 합이아닌 덮어 씌워진다.

In [ ]:
#14.2 해결책
# 클래스의 코드에 그대로 반영하면 된다.
class Variable:
  #생략
  def  backward(self):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = [self.creator]
    while funcs:
      f = funcs.pop()
      gys = [output.grad for output in f.outputs]
      gxs = f.backward(*gys)
      if not isinstance(gxs, tuple):
        gxs = (gxs, )

      for x, gx in zip(f.inputs, gxs):
        if x.grad is None:
          x.grad = gx
        else:
          x.grad = x.grad + gx
        
        if x.creator is not None:
          funcs.append(x.creator)

In [ ]:
#앞에서 실패했던 계산에 다시도전
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)
#실행결과 2.0

In [ ]:
#x를 세 번 사용해도 여전히 잘 작동하는가
x = Variable(np.array(3.0))
y = add(add(x, x), x)
y.backward()
print(x.grad)
#결과 3.0

In [ ]:
#14.3 미분값 재설정

x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)

# 두 번째 계산(같은 x를 사용하여 다른 계산을 수행)
y = add(add(x, x), x)
y.backward()
print(x.grad)

#실행 결과 
#2.0
#5.0

In [ ]:
class Variable:
  def cleargrad(self):
    self.grad = None

In [ ]:
# 첫 번째 계산
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad) # 2.0

# 두 번째 계산(같은 x를 사용하여 다른 계산을 수행)
x.cleargrad() #미분값 초기화
y = add(add(x, x), x)
y.backward()
print(x.grad) # 3.0

#실행결과 2.0 , 3.0

# Step 15 복잡한 계산 그래프(이론 편)

In [ ]:
#15.1 역전파의 올바른 순서
#15.2 현재의 DeZerp

class Variable:
  '''생략'''
  def backward(self):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = [self.creator]
    while funcs:
      f = funcs.pop()
      gys = [output.grad for output in f.outputs]
      gxs = f.backward(*gys)
      if not isinstance(gxs, tuple):
        gxs = (gxs, )
      
      for x, gx in zip(f.inputs, gxs):
        if x.grad is None:
          x.grad = gx
        else:
          x.grad = x.grad + gx
        
        if x.creator is not None:
          funcs.append(x.creator)

#15.3 함수 우선순위

# Step 16 복잡한 계산 그래프(구현 편)

In [ ]:
#16.1 세대 추가
#먼저 Variable 클래스와 Function 클래스에 인스턴스 변수 generation 을 추가한다.
#몇 번째 '세대'의 함수(혹은 변수)인지 나타내는 변수.

class Variable:
  def __init__ (self, data):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{}은(는) 지원하지 않습니다'.format(type(data)))

      self.data = data
      self.grad = None
      self.creator = None
      self.generation = 0 #세대 수를 기록하는 변수
  
  def set_creator(self,func):
    self.creator = func
    self.generation = func.generation + 1 # 세대를 기록한다(부모 세대 + 1)

In [ ]:
class Function(object):
  def __call__(self, *inputs):
    xs = [x.data for x in inputs]
    ys = self.forward(*xs)
    if not isinstance(ys, tuple):
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys]

  self.generation = max([x.generation for x in inputs])
  for output in outputs:
    output.set_creator(self)
  self.inputs = inputs
  self.outputs = outputs
  return outputs if len(outputs) > 1 else outputs[0]

In [ ]:
#16.2 세대 순으로 꺼내기
generations = [2, 0, 1, 4, 2]
funcs = []

for g in generations:
  f = Function()  # 더미 함수 클래스
  f.generation = g
  funcs.append(f)

[f.generation for f in funcs]
[2, 0, 1, 4, 2]

In [ ]:
funcs.sort(key=lambda x: x.generation)  #리스트 정렬
[f.generation for f in funcs]
[0, 1, 2, 2, 4]

f = funcs.pop()
f.generation

In [ ]:
#16.3 Variable 클래스의 backward
class Variable:
  '''생략'''
  def backward(self):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = []
    seen_set = set()

    def add_func(f):
      if f not in seen_set:
        funcs.append(f)
        seen_set.add(f)
        funcs.sort(key=lambda x: x.generation)

    add_func(self.creator)

    while funcs:
      f = funcs.pop()
      gys = [output.grad for output in f.outputs]
      gxs = f.backward(*gys)
      if not isinstance(gxs, tuple):
        gxs = (gxs,)
      
      for x, gx in zip(f.inputs, gxs):
        if x.grad is None:
          x.grad = gx
        else:
          x.grad = x.grad + gx

        if x.creator is not None:
          add_func(x.creator) # 수정 전 : funcs.append(x.creator)

In [ ]:
# 16.4 동작 확인
x = Variable(np.array(2.0))
a = square(x)
y = add(square(a), square(a))
y.backward()

print(y.data)
print(x.grad)

# 결과 32.0
#64.0

# Step 메모리 관리와 순환 참조

In [ ]:
#17.1 메모리 관리
#17.2 참조 카운트 방식의 메모리 관리